### 청약 매물 기사 크롤링 함수
- 내용에 '청약' 포함 기사만 크롤링
- 한 매물당 기사 크롤링 최대 개수는 10개로 제한
********************************************
- 매물 하나만 크롤링하고 싶다 -> 1~3번 수행
- 매물 데이터셋이 있고 크롤링하고 싶다 -> 4번 수행 
********************************************
- 4번 코드는 다음과 같이 작동함
    1. 아파트 이름과 "청약" 키워드를 조합하여 네이버 뉴스를 검색
    2. 검색 결과에서 "네이버 뉴스" 링크가 있는 기사만 선별
    3. 선별된 기사의 전문(full content)을 크롤링
    4. 크롤링된 기사 내용에 "청약" 키워드가 포함된 경우에만 저장
    5. 각 아파트당 최대 10개의 기사를 수집
    6. 이 과정을 모든 수도권 아파트에 대해 반복
    7. 최종 결과를 CSV 파일로 저장

In [52]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

1. 검색 URL 생성 함수 (아파트명 + 청약)

In [ ]:
import urllib.parse

def generate_news_url(apartment_name: str, apartment_ds, apartment_de) -> str:
    """아파트명 + 청약 검색 URL 생성"""
    base_url = "https://search.naver.com/search.naver"
    query = f'{apartment_name} 청약'  # 쌍따옴표 포함 검색
    
    params = {
        "where": "news",
        "query": query,
        "sm": "tab_opt",
        "sort": 0, # 관련도순 정렬
        "nso": f'so:r,p:from{apartment_ds}to{apartment_de}'  # 조회기간: 모집공고일 ~ 당첨자발표일
    }
    
    return base_url + "?" + urllib.parse.urlencode(params, quote_via=urllib.parse.quote)

2. 네이버 뉴스 전용 크롤링 함수

In [ ]:
import requests
from bs4 import BeautifulSoup

def crawl_naver_news(url: str, max_articles: int = 3) -> list:
    """청약 관련 기사만 필터링하는 네이버 뉴스 크롤러"""
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36",
        "Accept-Language": "ko-KR,ko;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Connection": "keep-alive",
        "Referer": "https://www.google.com/",
    }
    
    # 검색 결과 페이지 파싱
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 네이버 뉴스 링크 필터링
    news_links = [
        a['href'] for a in soup.select('a.info')
        if '네이버뉴스' in a.text and 'news.naver.com' in a['href']
    ][:max_articles]
    
    # 기사 본문 추출 + 청약 필터링
    articles = []
    for article_url in news_links:
        try:
            article_res = requests.get(article_url, headers=headers, timeout=5)
            article_soup = BeautifulSoup(article_res.text, 'html.parser')
            
            # 본문 추출
            content = article_soup.select_one('#newsct_article, #dic_area').get_text(strip=True)
            
            # 청약 키워드 필터링
            if '청약' not in content:
                print(f"제외: {article_url} (청약 미포함)")
                continue
                
            # 필수 정보 추출
            articles.append({
                "title": article_soup.select_one('#title_area').get_text(strip=True),
                "date": article_soup.select_one('.media_end_head_info_datestamp_time')['data-date-time'],
                "content": content,
                "url": article_url
            })
            
        except Exception as e:
            print(f"기사 처리 실패 ({article_url}): {str(e)}")
    
    return articles

3. 통합 실행

In [22]:
if __name__ == "__main__":
    # 아파트명 입력
    apartment_name = "래미안 원베일리"
    apartment_ds = '20240510'
    apartment_de = '20240522'
    
    # 1단계: 검색 URL 생성
    search_url = generate_news_url(apartment_name, apartment_ds, apartment_de)
    print(f"생성된 검색 URL: {search_url}")
    
    # 2단계: 네이버 뉴스 기사 크롤링
    news_data = crawl_naver_news(search_url)
    
    # 결과 출력
    print(f"\n{apartment_name} 관련 청약 뉴스 {len(news_data)}건")
    for idx, article in enumerate(news_data, 1):
        print(f"\n[{idx}] {article['title']}")
        print(f"작성일: {article['date']}")
        print(f"요약: {article['content']}")
        print(f"URL: {article['url']}")

생성된 검색 URL: https://search.naver.com/search.naver?where=news&query=%EB%9E%98%EB%AF%B8%EC%95%88%20%EC%9B%90%EB%B2%A0%EC%9D%BC%EB%A6%AC%20%EC%B2%AD%EC%95%BD&sm=tab_opt&sort=0&nso=so%3Ar%2Cp%3Afrom20240510to20240522

래미안 원베일리 관련 청약 뉴스 3건

[1] '20억 로또청약' 래미안 원베일리 1가구 모집에 3만5000명 '우르르'
작성일: 2024-05-21 08:06:04
요약: 28일 당첨자 발표, 내달 10~12일 계약서울 서초구 반포동 '래미안 원베일리'의 조합원 취소 물량 1가구 모집에 3만5000명이 몰렸다. /삼성물산당첨만 되면 20억원의 시세차익을 기대할 수 있는 서울 서초구 반포동 '래미안 원베일리' 조합원 취소분 1가구 청약에 3만5000여명이 몰렸다.20일 한국부동산원 청약홈에 따르면 이날 '래미안 원베일리(전용면적 84㎡)' 1가구에 대한 1순위 청약을 진행한 결과 총 3만5076명이 접수했다. 청약에 나온 가구는 계약 취소분이 아니라 조합원이 계약하지 않아 공급이 취소된 물량이다. 이에 소위 '줍줍'이라 불리는 무순위 청약 방식이 아닌 일반 분양 방식으로 공급됐다. 서울에 2년 이상 거주한 세대주만 1순위 청약 대상이며, 청약 통장을 보유해야 한다. 당첨자는 가점제로 뽑는다.청약자가 몰린 것은 3년 전 분양가가 적용된 탓이다. 이번에 공급된 전용 84㎡D 형의 공급금액은 19억5638만원이다. 국토교통부 실거래가 공개시스템에 따르면 해당 아파트의 같은 평형대 매물은 40~42억원에 거래되고 있다. 당첨만 되면 20억원 가량의 시세 차익을 기대할 수 있다.당첨자 발표일은 오는 28일이다. 계약일은 내달 10일부터 12일까지로, 이 시점까지 잔금을 치를 수 있어야 한다. 전매제한은 3년이지만 조합원 물량이라 실거거주 의무기간이 없어 바로 전세를 놓을 수 있다. 반포 래미안

4. 수도권 매물만 기사 크롤링

In [ ]:
import pandas as pd
import time
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import urllib.parse
import re

def generate_news_url(apartment_name, apartment_ds, apartment_de):
    base_url = "https://search.naver.com/search.naver"
    # 괄호와 그 안의 내용 제거
    apartment_name = re.sub(r'\([^)]*\)', '', apartment_name).strip()
    query = f'{apartment_name} 청약'
    params = {
        "where": "news",
        "query": query,
        "sm": "tab_opt",
        "sort": 0, # 관련도순 정렬
        "nso": f'so:r,p:from{apartment_ds}to{apartment_de}'  # 조회기간: 모집공고일 ~ 당첨자발표일
    }
    return base_url + "?" + urllib.parse.urlencode(params)

def crawl_naver_news(url, max_articles=3):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36",
        "Accept-Language": "ko-KR,ko;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Connection": "keep-alive",
        "Referer": "https://www.google.com/",
    }
    articles = []
    page = 1  # 페이지 번호 초기화

    cnt = 0
    
    while cnt < max_articles:  # 원하는 최대 기사 수에 도달할 때까지 반복
        try:
            # 페이지 URL 생성 (페이지 번호 적용)
            paged_url = f"{url}&start={(page - 1) * 10 + 1}"
            response = requests.get(paged_url, headers=headers)
            response.raise_for_status()  # HTTP 에러 확인

            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 기사 영역 선택
            news_areas = soup.select(".news_area")
            
            # 기사 영역이 없으면 종료
            if not news_areas:
                print("더 이상 기사 영역이 없습니다.")
                break

            for item in news_areas:
                # "네이버 뉴스" 버튼이 있는지 확인
                naver_news_link = item.select_one("a[href*='news.naver.com']")
                if not naver_news_link:
                    continue  # "네이버 뉴스" 링크가 없으면 다음 기사로 건너뜀
                
                title = item.select_one(".news_tit").text
                news_url = naver_news_link['href']  # 네이버 뉴스 URL 사용

                # 기사 본문 크롤링 함수 호출
                full_content = crawl_article_content(news_url, headers)

                # 키워드 필터링
                if "청약" in full_content:  # 크롤링된 전체 내용에서 "청약" 키워드 확인
                    articles.append({"title": title, "content": full_content, "url": news_url})

                cnt += 1
                print(f'{cnt}번 기사 - {title}')

                if (cnt >= max_articles):
                    break
            
            # 다음 페이지로 이동
            page += 1
            time.sleep(1)  # 페이지 요청 간 3초 대기

        except requests.exceptions.RequestException as e:
            print(f"요청 에러 발생: {e}")
            break  # 요청 에러 발생 시 크롤링 중단
        except Exception as e:
            print(f"파싱 에러 발생: {e}")
            break  # 파싱 에러 발생 시 크롤링 중단

    return articles

def crawl_article_content(news_url, headers):
    """기사 URL을 받아서 전체 내용을 크롤링하는 함수"""
    try:
        article_response = requests.get(news_url, headers=headers)
        article_response.raise_for_status()  # HTTP 에러 확인
        article_soup = BeautifulSoup(article_response.text, 'html.parser')

        # 네이버 뉴스 본문 요소 선택 (기사에 따라 선택자가 다를 수 있음)
        content_element = article_soup.select_one('#newsct_article') or article_soup.select_one('#dic_area')

        if content_element:
            return content_element.get_text(strip=True)
        else:
            return "기사 본문 내용을 찾을 수 없습니다."

    except requests.exceptions.RequestException as e:
        print(f"기사 내용 요청 에러 발생: {e}")
        return "기사 내용을 가져오는 데 실패했습니다."
    except Exception as e:
        print(f"기사 내용 파싱 에러 발생: {e}")
        return "기사 내용을 파싱하는 데 실패했습니다."

if __name__ == "__main__":
    # 데이터셋 로드
    df = pd.read_csv('../storage/train_data/train-250315-01.csv', encoding='euc-kr')

    # 수도권 필터링
    metropolitan_areas = ['서울', '경기', '인천']
    metropolitan_df = df[df['공급지역명'].isin(metropolitan_areas)]

    # 중복 제거
    metropolitan_df = metropolitan_df.drop_duplicates(subset="공고번호", keep="first").reset_index(drop=True)

    # 크롤링 할 매물 ids
    df_ids = metropolitan_df['공고번호'].tolist()

    # 결과를 저장할 리스트
    all_news_data = []

    # 각 아파트에 대해 뉴스 크롤링
    for id in df_ids:
        # 0단계: 아파트 정보 추출
        df_apartment = metropolitan_df[metropolitan_df['공고번호'] == id].iloc[0]
        apartment_name = df_apartment['주택명']
        apartment_ds = df_apartment['모집공고일'].replace('-', '')
        apartment_de = df_apartment['당첨자발표일'].replace('-', '')

        print(f'{id} - {apartment_name} 크롤링')

        # 1단계: 검색 URL 생성
        search_url = generate_news_url(apartment_name, apartment_ds, apartment_de)
        
        # 2단계: 네이버 뉴스 기사 크롤링 (최대 10개)
        news_data = crawl_naver_news(search_url, max_articles=3)  # 여기를 수정

        # 결과 저장
        for article in news_data:
            article['공고번호'] = id
            article['apartment'] = apartment_name
        all_news_data.extend(news_data)
        
        # 과도한 요청 방지를 위한 대기
        time.sleep(1)

    # 결과를 DataFrame으로 변환
    result_df = pd.DataFrame(all_news_data)

    # CSV 파일로 저장
    result_df.to_csv('수도권_아파트_청약_뉴스_크롤링_청약기간내.csv', index=False, encoding='utf-8-sig')
    print(f"총 {len(result_df)}개의 기사가 크롤링되어 CSV 파일로 저장되었습니다.")

2020001374 - 봉담자이 라피네 크롤링
1번 기사 - 화성 봉담도 '청약 열풍'···가점 50점 넘어야 당첨
2번 기사 - GS건설, 화성 '봉담자이 라피네' 청약 평균 22.09대 1…1순위 마감
3번 기사 - 화성 `향남역 한양수자인 디에스티지`, 1순위 청약 시작
2020001419 - 포천 송우 1 서희스타힐스 크롤링
1번 기사 - 오늘 e편한세상 영종 센텀베뉴 청약 접수
더 이상 기사 영역이 없습니다.
2020001389 - 힐스테이트 용인 둔전역 크롤링
1번 기사 - ‘힐스테이트 용인 둔전역’ 청약 평균 6대 1로 마감
2번 기사 - ‘힐스테이트 용인 둔전역’ 최고경쟁률 25대 1
3번 기사 - 현대건설 ‘힐스테이트 용인 둔전역’ 경쟁률 25대 1 기록
2020001447 - 더샵 지제역 센트럴파크2BL 크롤링
더 이상 기사 영역이 없습니다.
2020001464 - 송도자이 크리스탈오션 크롤링
1번 기사 - 송도자이 크리스탈오션, 1순위 청약 경쟁률 ‘29대1’
2번 기사 - 인천 송도자이크리스탈오션 1순위 청약경쟁률 21대 1
3번 기사 - [분양캘린더] '송도자이 크리스탈오션' 28일 청약
2020001451 - 의정부 고산 수자인 디에스티지 C4BL 크롤링
1번 기사 - 한양, '의정부 고산 수자인 디에스티지' 19일 1순위 청약
더 이상 기사 영역이 없습니다.
2020001450 - 의정부 고산 수자인 디에스티지 C3BL 크롤링
더 이상 기사 영역이 없습니다.
2020001449 - 의정부 고산 수자인 디에스티지 C1BL 크롤링
더 이상 기사 영역이 없습니다.
2020001394 - 한화 포레나 인천연수 크롤링
1번 기사 - 한화 ‘포레나 장안’ 등 올 2만1629가구 공급
2번 기사 - 한화건설, 올해 전국에 2만1629가구 '포레나' 공급
3번 기사 - 발코니 확장 통해 수납공간 강화한 `한화 포레나 인천연수` 19일 1순위 청...
2020001360 - e편한세상 부평 그랑힐스 크롤링
1번 기사 - ‘e편한세상 부평 그랑힐스’ 

In [96]:
df = pd.read_csv('수도권_아파트_청약_뉴스_크롤링_청약기간내.csv')
df

,공고번호,apartment,title,content,url
0,2025000043,부천 JY 포에시아,'안동용상하늘채리버스카이' 등 1205가구 분양예정[분양캘린더],견본주택 '용현우방아이유쉘센트럴마린' 개관[서울=뉴시스][서울=뉴시스]정진형 기자 ...,https://n.news.naver.com/mnews/article/003/001...
1,2025000043,부천 JY 포에시아,다음 주 전국 분양물량 1205가구에 그쳐…서울은 '無',다음 주 분양 물량이 전주의 절반에도 미치지 못하는 1200여가구에 그칠 것으로 보...,https://n.news.naver.com/mnews/article/277/000...
2,2025000043,부천 JY 포에시아,분양 성수기인데…내주 전국서 1205가구 공급 그쳐,부동산R114 주간 부동산 분양 캘린더봄철 분양 성수기에도 분양 시장은 침체기가 이...,https://n.news.naver.com/mnews/article/015/000...
3,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),"3월 첫째주, 전국 2334가구 청약…지방 물량 집중",전국 6곳 청약 접수서울·인천 수도권 2곳3월 첫째 주 전국 6곳에서 총 2334가...,https://n.news.naver.com/mnews/article/277/000...
4,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),[주간분양] 수도권 청약 물량 희귀…전국 2334가구 중 35가구,전국 6곳 중 4곳이 지방…2299가구 공급28일 리얼투데이에 따르면 3월 첫째 주...,https://n.news.naver.com/mnews/article/119/000...
...,...,...,...,...,...
1948,2020000312,르엘 신반포,“현금부자 몰렸다”…르엘신반포 청약 당첨가점 최고 74점,"커트라인 62점, 매매시장 침체에도 청약열기 고조10억원 안팎 시세차익 기대감 현금...",https://n.news.naver.com/mnews/article/016/000...
1949,2020000312,르엘 신반포,코로나19에도 식을 줄 모르는 '청약 광풍'…왜?,"""당첨되면 로또""…규제지역서도 청약경쟁률 수백 대 일 껑충규제·비(非)규제 모두 청...",https://n.news.naver.com/mnews/article/003/000...
1950,2020000298,시흥장현 영무예다음,최고 경쟁률 기록한 '시흥장현영무예다음'···커트라인도 高高,[서울경제] 2·20 대책에 따른 2차 풍선효과 혜택을 보고 있는 시흥시에서 청약 ...,https://n.news.naver.com/mnews/article/011/000...
1951,2020000298,시흥장현 영무예다음,경기 비규제지역 '안시성' 청약률 '高高',아파트값 상대적으로 낮고청약조건 까다롭지 않아 인기정부의 잇단 규제 대책에6억 이하...,https://n.news.naver.com/mnews/article/015/000...
